<a href="https://colab.research.google.com/github/ahorvath008/Coursera_Capstone/blob/main/Toronto_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>